# APRENDENDO PANDAS

* Import Fase

In [76]:

import pandas as pd
import requests
from bs4 import BeautifulSoup



* Request Fase

In [77]:
# Request web page
resp = requests.get("https://www.fundsexplorer.com.br/ranking")

# Get html content
html = resp.text


* Parser Functions

In [78]:


def tableDataText(table):
    """Parses a html segment started with tag <table> followed
    by multiple <tr> (table rows) and inner <td> (table data) tags.
    It returns a list of rows with inner columns.
    Accepts only one <th> (table header/data) in the first row.
    """

    def rowgetDataText(tr, coltag="td"):  # td (data) or th (header)
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]

    rows = []
    trs = table.find_all("tr")
    headerow = rowgetDataText(trs[0], "th")
    if headerow:  # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs:  # for every table row
        rows.append(rowgetDataText(tr, "td"))  # data row
    return rows


def parser_text(text: str):
    """ Parser string text e.g R$ 99,01 to 99.01"""
    if text == "N/A":
        return 0

    text = str(text).replace(" ", "")
    text = str(text).replace("(", "")
    text = str(text).replace(")", "")
    text = str(text).replace("R$", "")
    text = str(text).replace("%", "")
    text = str(text).replace(".", "")
    text = str(text).replace(",", ".")
    text = str(text).replace("/", "")
    try:
        return float(text)
    except:
        return text



* Soap Fase

In [79]:

# Create soap instance
soup = BeautifulSoup(html, features="html.parser")

# Find table in html content
htmltable = soup.find("table", {"id": "table-ranking"})
table = tableDataText(htmltable)


* Carregando DataFrame

In [80]:
# Parseamentos dos titulos das colunas
colunas = map(parser_text, table[0])

# Iniciando dataframe
df = pd.DataFrame(table[1:], columns=colunas)



# Operações com dataframe

In [81]:

# Parsear varias colunas via callback com apply
for coluna in df.columns:
    df.loc[:, coluna] = df[coluna].apply(parser_text)


In [82]:

# Buscar com condicao em coluna expecifica
df.loc[df["Códigodofundo"] == "MXRF11"]


,Códigodofundo,Setor,PreçoAtual,LiquidezDiária,Dividendo,DividendYield,DY3MAcumulado,DY6MAcumulado,DY12MAcumulado,DY3MMédia,...,PatrimônioLíq,VPA,PVPA,DYPatrimonial,VariaçãoPatrimonial,RentabPatrnoPeríodo,RentabPatrAcumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
74,MXRF11,Híbrido,9.6,8183060.0,0.11,1.11,3.31,6.17,10.84,1.1,...,2.302694e+09,10.19,0.94,0.98,-0.02,0.96,2.22,0.0,0.0,0.0


In [83]:

# Buscar com condicao com colunas expecificas
# Exibindo resposta com colunas expessificas passadas via lista
df.loc[df["Códigodofundo"] == "MXRF11", ["Códigodofundo", "PreçoAtual"]]


,Códigodofundo,PreçoAtual
74,MXRF11,9.6


In [84]:

# Converter tipo de dados da coluna
# df.loc[:, "DY12MMédia"] = df["DY12MMédia"].astype(float)
# df.loc[:, "PVPA"] = df["PVPA"].astype(float)
# df.loc[:, "QuantidadeAtivos"] = df["QuantidadeAtivos"].astype(float)
# df.loc[:, "VacânciaFísica"] = df["VacânciaFísica"].astype(float)


In [85]:

# Query
colunas = [
    "Códigodofundo",
    "Setor",
    "PreçoAtual",
    "DividendYield",
    "DY12MMédia",
    "RentabAcumulada",
    "PatrimônioLíq",
    "PVPA",
    "VacânciaFísica",
    "VacânciaFinanceira",
    "QuantidadeAtivos",
]
pesquisa = df.query("DY12MMédia > 0.5 & QuantidadeAtivos > 10 & VacânciaFísica < 10")
pesquisa = pesquisa.loc[:,colunas].sort_values(
    ["DY12MMédia", "DividendYield"], ascending=False
)
pesquisa


,Códigodofundo,Setor,PreçoAtual,DividendYield,DY12MMédia,RentabAcumulada,PatrimônioLíq,PVPA,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
111,BNFS11,Outros,119.49,1.19,1.00,-0.28,6.941729e+07,1.20,0.00,0.00,18.0
102,BBPO11,LajesCorporativas,80.02,1.06,0.95,-6.17,1.512932e+09,0.84,0.00,0.00,64.0
170,RBVA11,Outros,89.87,0.99,0.91,-1.48,1.193289e+09,0.87,0.70,0.00,72.0
95,BBRC11,Outros,99.94,0.99,0.88,2.60,1.703974e+08,0.93,5.37,1.08,20.0
131,TRXF11,Outros,98.61,0.86,0.77,1.44,7.081472e+08,1.01,0.00,0.00,44.0
185,QAGR11,Outros,47.12,0.84,0.77,-1.48,3.008317e+08,0.79,0.00,0.00,12.0
189,GGRC11,Logística,107.00,0.85,0.75,-2.35,1.012027e+09,0.82,0.00,0.00,15.0
177,HGLG11,Logística,162.80,0.67,0.75,0.30,3.501808e+09,1.09,6.90,6.50,17.0
266,BTLG11,Logística,100.00,0.71,0.69,2.94,1.504876e+09,1.01,0.00,2.00,16.0
179,HGRU11,Híbrido,116.72,0.69,0.67,6.77,2.191214e+09,0.98,0.00,0.00,17.0


In [86]:

# Retorna lista de valores unicos em uma coluna
df.loc[:, "Setor"].unique()


array(['Shoppings', 'TítuloseValMob', 'LajesCorporativas', 'Híbrido',
       'Hospital', 'Logística', '', 'Hotel', 'Residencial', 'Outros'],
      dtype=object)

In [87]:

# Metodo de iterecao
for row in pesquisa.itertuples():
    print('',end='\n')
    for index, column in enumerate(pesquisa.columns):
        print(column,row[index + 1],end=', ')
    break


Códigodofundo BNFS11, Setor Outros, PreçoAtual 119.49, DividendYield 1.19, DY12MMédia 1.0, RentabAcumulada -0.28, PatrimônioLíq 69417286.55, PVPA 1.2, VacânciaFísica 0.0, VacânciaFinanceira 0.0, QuantidadeAtivos 18.0, 

In [88]:


# Gravar no excel
pesquisa.to_excel("output.xlsx")
